# packages used on conda enviroment

In [1]:
conda list

/bin/bash: /home/anish/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

# packages in environment at /home/anish/miniconda3/envs/dlenv_conda:

#

# Name                    Version                   Build  Channel

_libgcc_mutex             0.1                        main  

_openmp_mutex             5.1                       1_gnu  

absl-py                   1.3.0                    pypi_0    pypi

anyio                     3.6.2                    pypi_0    pypi

argon2-cffi               21.3.0                   pypi_0    pypi

argon2-cffi-bindings      21.2.0                   pypi_0    pypi

arrow                     1.2.3                    pypi_0    pypi

asttokens                 2.2.1                    pypi_0    pypi

astunparse                1.6.3                    pypi_0    pypi

attrs                     22.2.0                   pypi_0    pypi

backcall                  0.2.0                    pypi_0    pypi

beautifulsoup4        

# Necessary Modules

In [2]:
import os
import zipfile

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout, concatenate 
from keras.losses import categorical_crossentropy
from keras.utils.vis_utils import plot_model
from keras import callbacks
     
import warnings
warnings.filterwarnings("ignore")

## Presence of GPU Checked

In [3]:
physical_devices = tf.config.experimental.list_logical_devices("GPU")
print(True if len(physical_devices) > 0 else False)

True


2023-01-04 21:07:28.403690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 21:07:28.404606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 21:07:28.550264: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 21:07:28.551192: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 21:07:28.551996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

### specified data path

In [4]:
kaggle_dir = '/kaggle/input/'
data_folder = 'mura-version-11'

data_path = os.path.join(kaggle_dir, data_folder)

In [5]:
def images_dataframe(csv: str) -> pd.DataFrame:
    
    global n_classes
    n_classes = 7
    
    images_data = pd.read_csv(os.path.join(data_path + "/MURA-v1.1", csv), dtype = str, header = None)
    images_data.columns = ['image_path']

    images_data['label'] = images_data['image_path'].apply(lambda x: "positive" if "positive" in x.split('/')[4].replace('study' + ('1' or '2'), '') else "negative")
    images_data['category']  = images_data['image_path'].apply(lambda x: x.split('/')[2])
    images_data['patient_Id']  = images_data['image_path'].apply(lambda x: x.split('/')[3].replace('patient',''))
    
    mapping = {'XR_SHOULDER': 'shoulder', 
               'XR_HUMERUS': 'humerus', 
               'XR_FINGER': 'finger', 
               'XR_ELBOW': 'elbow',
               'XR_WRIST': 'wrist',
               'XR_FOREARM': 'forearm', 
               'XR_HAND': 'hand'}
    
    images_data['category'].replace(mapping, inplace = True)
    images_data["category"] = images_data["category"].apply(lambda x: x.split(","))
    
    return images_data

In [6]:
train_data = images_dataframe("train_image_paths.csv")
valid_data = images_dataframe("valid_image_paths.csv")

In [7]:
def data_analysis(df: pd.DataFrame) -> None:
    
    print(f"The shape of the Dataframe :\n\n {df.shape}\n")
    print(f"Total negative and positive case :\n\n{df['label'].value_counts(dropna = False, sort = False)}\n")
    print(f"Total number of body parts :\n\n{df['category'].value_counts(dropna = False, sort = False)}\n")
    
    print(f"Null values present in dataframe :\n\n{df.isnull().any()}")

In [8]:
train_data

,image_path,label,category,patient_Id
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive,[shoulder],00001
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive,[shoulder],00001
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,positive,[shoulder],00001
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive,[shoulder],00002
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,positive,[shoulder],00002
...,...,...,...,...
36803,MURA-v1.1/train/XR_HAND/patient11183/study1_ne...,negative,[hand],11183
36804,MURA-v1.1/train/XR_HAND/patient11183/study1_ne...,negative,[hand],11183
36805,MURA-v1.1/train/XR_HAND/patient11184/study1_ne...,negative,[hand],11184
36806,MURA-v1.1/train/XR_HAND/patient11184/study1_ne...,negative,[hand],11184


In [9]:
data_analysis(train_data)

The shape of the Dataframe :

 (36808, 4)

Total negative and positive case :

positive    14873
negative    21935
Name: label, dtype: int64

Total number of body parts :

[shoulder]    8379
[humerus]     1272
[finger]      5106
[elbow]       4931
[wrist]       9752
[forearm]     1825
[hand]        5543
Name: category, dtype: int64

Null values present in dataframe :

image_path    False
label         False
category      False
patient_Id    False
dtype: bool


In [10]:
valid_data

,image_path,label,category,patient_Id
0,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...,positive,[wrist],11185
1,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...,positive,[wrist],11185
2,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...,positive,[wrist],11185
3,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...,positive,[wrist],11185
4,MURA-v1.1/valid/XR_WRIST/patient11186/study1_p...,positive,[wrist],11186
...,...,...,...,...
3192,MURA-v1.1/valid/XR_FINGER/patient11967/study1_...,negative,[finger],11967
3193,MURA-v1.1/valid/XR_FINGER/patient11967/study1_...,negative,[finger],11967
3194,MURA-v1.1/valid/XR_FINGER/patient11738/study1_...,negative,[finger],11738
3195,MURA-v1.1/valid/XR_FINGER/patient11738/study1_...,negative,[finger],11738


In [11]:
data_analysis(valid_data)

The shape of the Dataframe :

 (3197, 4)

Total negative and positive case :

positive    1530
negative    1667
Name: label, dtype: int64

Total number of body parts :

[wrist]       659
[forearm]     301
[hand]        460
[humerus]     288
[shoulder]    563
[elbow]       465
[finger]      461
Name: category, dtype: int64

Null values present in dataframe :

image_path    False
label         False
category      False
patient_Id    False
dtype: bool


In [12]:
train_data_gen_args = dict(samplewise_center = True,
                     samplewise_std_normalization = True,
                     rotation_range = 5,
                     width_shift_range = 0.1,
                     height_shift_range = 0.1,
                     zoom_range = 0.2,
                     horizontal_flip = True,
                     vertical_flip = True)

train_image_datagen = ImageDataGenerator(**train_data_gen_args)

valid_data_gen_args = dict(samplewise_center = True,
                     samplewise_std_normalization = True,
                     rotation_range = 5,
                     width_shift_range = 0.1,
                     height_shift_range = 0.1,
                     zoom_range = 0.2,
                     horizontal_flip = False,
                     vertical_flip = False)

valid_image_datagen = ImageDataGenerator(**valid_data_gen_args)

In [13]:
image_path = '/kaggle/input/mura-version-11/'
output_path = "/kaggle/working/train_test_images"
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
train_image_generator = train_image_datagen.flow_from_dataframe(
    dataframe = train_data,
    directory = image_path,
    x_col = 'image_path',
    y_col = 'category',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = "categorical",
    classes = ['shoulder',
               'humerus', 
               'finger', 
               'elbow',
               'wrist',
               'forearm', 
               'hand'],
    validate_filenames = False,
    target_size = (224, 224),
    #save_to_dir = output_path,
    save_format = "png",
    color_mode = "rgb")

valid_image_generator = valid_image_datagen.flow_from_dataframe(
    dataframe = valid_data,
    directory = image_path,
    x_col = 'image_path',
    y_col = 'category',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = "categorical",
    classes = ['shoulder',
               'humerus', 
               'finger', 
               'elbow',
               'wrist',
               'forearm', 
               'hand'],
    validate_filenames = False,
    target_size = (224, 224),
    #save_to_dir = output_path,
    save_format = "png",
    color_mode = "rgb",)

Found 36808 non-validated image filenames belonging to 7 classes.
Found 3197 non-validated image filenames belonging to 7 classes.


In [14]:
train_images_batch, train_labels = next(train_image_generator)
validation_images_batch, validation_labels = next(valid_image_generator)

def patch_result(images_batch, labels, generator, train):
    
    if train:
        print("training input images patch shape  : ", images_batch.shape)
        print("training input labels patch shape  : ", labels.shape)
        print("training labels:", generator.class_indices)
    else:
        print("validation input images patch shape: ", images_batch.shape)
        print("validation input labels patch shape: ", labels.shape)
        print("validation labels:", generator.class_indices)

In [15]:
patch_result(train_images_batch, train_labels, train_image_generator, train = True)

training input images patch shape  :  (32, 224, 224, 3)
training input labels patch shape  :  (32, 7)
training labels: {'elbow': 0, 'finger': 1, 'forearm': 2, 'hand': 3, 'humerus': 4, 'shoulder': 5, 'wrist': 6}


In [16]:
patch_result(validation_images_batch, validation_labels, valid_image_generator, train = False)

validation input images patch shape:  (32, 224, 224, 3)
validation input labels patch shape:  (32, 7)
validation labels: {'elbow': 0, 'finger': 1, 'forearm': 2, 'hand': 3, 'humerus': 4, 'shoulder': 5, 'wrist': 6}


# Model Creation

## GoogleNet

> Creation of Inception block

In [17]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4):
    
    # Input: 
    # - f1: number of filters of the 1x1 convolutional layer in the first path
    # - f2_conv1, f2_conv3 are number of filters corresponding to the 1x1 and 3x3 convolutional layers in the second path
    # - f3_conv1, f3_conv5 are the number of filters corresponding to the 1x1 and 5x5  convolutional layer in the third path
    # - f4: number of filters of the 1x1 convolutional layer in the fourth path

    # 1st path:
    path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

    # 2nd path
    path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
    path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

    # 3rd path
    path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
    path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

    # 4th path
    path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
    path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

    output_layer = concatenate([path1, path2, path3, path4], axis = -1)

    return output_layer

> Creation of Layers

In [18]:
def layer_1(input_layer):
    
    '''composed of :
    > convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
    > max-pooling layer: pool_size = (3,3), strides = 2'''
    
    X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)
    
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)
    
    return X

def layer_2(X):
    
    '''composed of :
    > convolutional layer: filters = 64, strides = 1
    > convolutional layer: filters = 192, kernel_size = (3,3)
    > max-pooling layer: pool_size = (3,3), strides = 2'''
    
    X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)
    X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)
    
    X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

    return X

def layer_3(X):
    
    '''composed of :
    > 1st Inception block
    > 2nd Inception block
    > max-pooling layer: pool_size = (3,3), strides = 2'''
    
    X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)
    X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)
    
    X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

    return X

def layer_4(X):
    
    '''composed of :
    > 3rd Inception block
    > Extra Network 1
    > 4th Inception block
    > 5th Inception block
    > 6th Inception block
    > Extra Network 2
    > 7th Inception block
    > max-pooling layer: pool_size = (3,3), strides = 2'''
    
    X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)
    
    X1 = Extra_Network1(X)
    
    X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)
    X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)
    X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)
    
    X2 = Extra_Network2(X)
    
    X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)
    
    X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

    return X, X1, X2

def layer_5(X):
    
    '''composed of :
    > 8th Inception block
    > 9th Inception block
    > Global Average pooling layer
    > Dropoutlayer'''
    
    X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)
    X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

    X = GlobalAveragePooling2D(name = 'GAPL')(X)
    
    X = Dropout(0.4)(X)
    
    return X

> Creation of Extra Networks

In [19]:
def Extra_Network1(X):
    
    X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
    X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
    X1 = Flatten()(X1)
    X1 = Dense(1024, activation = 'relu')(X1)
    X1 = Dropout(0.7)(X1)
    X1 = Dense(n_classes, activation = 'softmax', name = 'output_1')(X1)
    
    return X1

def Extra_Network2(X):
    
    X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
    X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
    X2 = Flatten()(X2)
    X2 = Dense(1024, activation = 'relu')(X2)
    X2 = Dropout(0.7)(X2)
    X2 = Dense(n_classes, activation = 'softmax', name = 'output_2')(X2)
    
    return X2

> Creation of GoogleNet

In [20]:
def _GoogleNet():
    
    # input layer 
    input_layer = Input(shape = (224, 224, 3))
    
    #architecture broken into layers
    X = layer_1(input_layer)
    X = layer_2(X)
    X = layer_3(X)
    
    X, X1, X2 = layer_4(X)
    
    X = layer_5(X)
    
    # output layer 
    X = Dense(n_classes, activation = 'softmax', name = 'output_3')(X)
    
    # model
    model = Model(inputs = input_layer,  outputs = [X, X1, X2], name = 'GoogLeNet')

    return model

In [21]:
_GoogleNet().summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 109, 109, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 54, 54, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 54, 54, 64)   4160        max_pooling2d[0][0]              
__________________________________________________________________________________________

In [22]:
_GoogleNet().plot_model()

AttributeError: 'Functional' object has no attribute 'plot_model'

In [37]:
EPOCH = 15
model_checkpoint_filepath = '/kaggle/temp/' 

def scheduler(epoch, lr):
    if epoch < 6:
        return lr
    else:
        return lr*tf.math.exp(-0.1)
  

def CallBack(model_name):
    
    global checkpoint_filepath
    
    if model_name == 'GoogleNet':
        lr = 0.00001

        Reduce_LR_Plateau_callback = callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.1,
            patience=1,
            verbose=0,
            mode="auto",
            min_delta=0.0001,
            cooldown=0,
            min_lr=0,
        )

        LR_scheduler_callback = callbacks.LearningRateScheduler(lambda EPOCH, lr: scheduler(EPOCH, lr), verbose = 0)

        early_stopping_callback = callbacks.EarlyStopping(
            monitor = "al_loss",
            min_delta = 0,
            patience=0,
            verbose=0,
            mode="auto",
            baseline = None,
            restore_best_weights = True,
            #start_from_epoch = 0
        )
                                                                
        try:
            checkpoint_filepath = os.path.join(model_checkpoint_filepath, model_name + '_model_weights.hdf5')
        except:
            pass
        
        model_checkpoint_callback = callbacks.ModelCheckpoint(
            filepath = checkpoint_filepath,
            save_weights_only=True,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True
        )
        
        return [Reduce_LR_Plateau_callback, LR_scheduler_callback, early_stopping_callback, model_checkpoint_callback]

_callbacks = CallBack('GoogleNet')

In [32]:
def CompileModel(name, model):
    
    if name == 'GoogleNet':
        model.compile(optimizer = 'adam', 
                      loss = categorical_crossentropy, 
                      metrics={"output_1":"accuracy",
                               "output_2":"accuracy",
                               "output_3":"accuracy"})
        
    return model

In [33]:
def FitModel(name, model):

    if name == 'GoogleNet':
        history = model.fit(train_image_generator,
                            epochs = EPOCH,
                            callbacks = CallBack('GoogleNet'),
                            validation_data = valid_image_generator,
                            steps_per_epoch = (int(len(train_image_generator.labels) / 160)),
                            validation_steps = (int(len(valid_image_generator.labels) / 160)))
        
        model.load_weights(checkpoint_filepath)
    
    return history, model
        

In [34]:
def BuildModel(name):
    
    if name == 'GoogleNet':
        prepared_model = _GoogleNet() 
        
    compiled_model = CompileModel(name, prepared_model)
    
    return compiled_model


In [38]:
compiled_model = BuildModel('GoogleNet')
model, history = FitModel('GoogleNet', compiled_model)

2023-01-04 21:12:10.811030: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2023-01-04 21:12:16.435307: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


230/230 [==============================] - 234s 952ms/step - loss: 4.9136 - output_3_loss: 1.6580 - output_1_loss: 1.6075 - output_2_loss: 1.6481 - output_3_accuracy: 0.3568 - output_1_accuracy: 0.3795 - output_2_accuracy: 0.3598 - val_loss: 4.5042 - val_output_3_loss: 1.5228 - val_output_1_loss: 1.4616 - val_output_2_loss: 1.5198 - val_output_3_accuracy: 0.4128 - val_output_1_accuracy: 0.4342 - val_output_2_accuracy: 0.4260
Epoch 2/15
230/230 [==============================] - 196s 849ms/step - loss: 3.8721 - output_3_loss: 1.3209 - output_1_loss: 1.2290 - output_2_loss: 1.3222 - output_3_accuracy: 0.4875 - output_1_accuracy: 0.5451 - output_2_accuracy: 0.4891 - val_loss: 3.9522 - val_output_3_loss: 1.3879 - val_output_1_loss: 1.2103 - val_output_2_loss: 1.3539 - val_output_3_accuracy: 0.5132 - val_output_1_accuracy: 0.5345 - val_output_2_accuracy: 0.5115
Epoch 3/15
230/230 [==============================] - 169s 735ms/step - loss: 3.1940 - output_3_loss: 1.0987 - output_1_loss: 1.015

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/kaggle/temp/GoogleNet_model_weights.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)